### 1. 모듈

In [1]:
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")

In [2]:
from selenium import webdriver
from random import randint
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import pandas as pd
import re

### 2. url 접속

In [18]:
driver = webdriver.Chrome('/Users/harampark/Documents/chromedriver')
url = 'http://likms.assembly.go.kr/bill/main.do'
driver.get(url)

In [19]:
search = driver.find_element_by_css_selector('li.gnbL2')
search.click()
time.sleep(2)

# driver.find_element_by_css_selector('#AGE_AREA > select:nth-child(2) > option:nth-child(2)').click()
# driver.find_element_by_css_selector('#AGE_AREA > select:nth-child(3) > option:nth-child(2)').click()

search = driver.find_element_by_css_selector('div.mt20.alignC > button.btnd')
search .click()
time.sleep(2)

more_bill = driver.find_element_by_css_selector('#pageSizeOption > option:nth-child(4)').click()

### 3. 크롤링

In [20]:
num_list = []
handled_list = []
title_list = []
proposer_type_list = []
propose_date_list = []
resolution_date_list = []
resolution_result_list = []
progress_list = []

proposer_list = []
paper_list = []
session_list = []
reason_list = []
board_list = []

In [21]:
n_times = 0

## 원하는 시작페이지
n_iter = 0

driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(11)').click()
while n_times <= n_iter - 2:
    driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(13)').click()
    n_times +=1

In [23]:
if n_iter == 0:
    start = 1; end = 11
else:
    start = 3; end = 13
    
## 10페이지가 끝나면 다음 10페이지를 불러옵니다.
while n_iter <= 17:

    ## start번째 페이지에 접속합니다.
    for k in tqdm(range(start,end)):
        driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({k})').click()
        ## start번째 페이지의 i번째 의안에 접근합니다.
        for i in range(1,101):
            ## i 번쨰 의안에 접근합니다.
            bill = driver.find_element_by_css_selector(f'body > div > div.contentWrap > div.subContents > div > div.tableCol01 > table > tbody > tr:nth-child({i})')

            num = bill.find_element_by_css_selector('td:nth-child(1)').text
            title = bill.find_element_by_css_selector('td.alignL').text
            proposer_type = bill.find_element_by_css_selector('td:nth-child(3)').text
            propose_date = bill.find_element_by_css_selector('td:nth-child(4)').text
            resolution_date = bill.find_element_by_css_selector('td:nth-child(5)').text
            resolution_result = bill.find_element_by_css_selector('td:nth-child(6)').text
            progress = bill.find_element_by_css_selector('td:nth-child(8)').text

            num_list.append(num)
            if resolution_date == '':
                handled_list.append('계')
            else:
                handled_list.append('처')
            title_list.append(title)
            proposer_type_list.append(proposer_type)
            propose_date_list.append(propose_date)
            resolution_date_list.append(resolution_date)
            resolution_result_list.append(resolution_result)
            progress_list.append(progress)


            ## start번째 페이지의 i번째 의안에 접속합니다.
            bill = driver.find_element_by_css_selector(f'body > div > div.contentWrap > div.subContents > div > div.tableCol01 > table > tbody > tr:nth-child({i}) > td.alignL > div.pl25 > a')
            bill.click()
            
            MAX_SLEEP_TIME = 3
            rand_value = randint(1, MAX_SLEEP_TIME)
            time.sleep(rand_value)
            paper = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(4)').text
            session = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(5)').text     

            paper_list.append(paper)
            session_list.append(session)
            
            ## 제안이유를 담습니다.
            try:
                # 더 보기를 클릭합니다.
                more_info = driver.find_element_by_css_selector('#summaryContentMoreBtn').click()
                reason_raw_text = driver.find_element_by_css_selector('div.contIn > div.textType02').text
                
                content = []
                reason_split = re.split('\r|\t|\n|\xa0', reason_raw_text)
                content.append(''.join(reason_split))
                reason_list.append(content)            
                
            except:
                reason_list.append([]) 

            ## 소관위원회를 담습니다.
            try:
                board = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div:nth-child(5) > div > table > tbody > tr > td:nth-child(1)').text
                board_list.append(board)
            except:
                board_list.append([])
            
            ## 제안자를 담습니다.
            member_list = []    
            try:
                proposer = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(3) > a > img')
                proposer.click()

                # 새로운 창으로 초점을 바꿉니다.
                driver.switch_to_window(driver.window_handles[1])
                driver.get_window_position(driver.window_handles[1])

                member_num = len(driver.find_elements_by_tag_name('a'))
                member_end = (member_num//3) + member_num + 2

                for j in range(2, member_end):
                    try:
                        member = driver.find_element_by_css_selector(f'#periodDiv > div.layerInScroll.coaTxtScroll > div > a:nth-child({j})').text
                        member_list.append(member)
                    except:
                        continue
                        
                driver.close()
                proposer_list.append(member_list)

                # 원래 창으로 초점을 바꿉니다.
                driver.switch_to_window(driver.window_handles[0])
                driver.get_window_position(driver.window_handles[0])

            except:
                proposer_list.append([])
    
            driver.back()
            driver.execute_script("window.scrollTo(0, window.scrollY + 70)")

    driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({10+start})').click()
    if n_iter == 0:
        start = 1; end = 11
    else:
        start = 3; end = 13
    n_iter += 1
    MAX_SLEEP_TIME = 6
    rand_value = randint(4, MAX_SLEEP_TIME)
    time.sleep(rand_value)

  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-23-ae90ee70a654>:79: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])
/usr/local/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py:1149: UserWarning: Only 'current' window is supported for W3C compatibile browsers.
  warnings.warn("Only 'current' window is supported for W3C compatibile browsers.")
<ipython-input-23-ae90ee70a654>:96: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[0])
 20%|██        | 2/10 [14:11<56:46, 425.84s/it]


ElementNotInteractableException: Message: element not interactable: element has zero size
  (Session info: chrome=84.0.4147.105)


### 4. 데이터프레임

In [25]:
result = [num_list, handled_list, title_list, proposer_type_list, propose_date_list, resolution_date_list, resolution_result_list,\
          progress_list, paper_list, session_list, reason_list, board_list, proposer_list]
result_df = pd.DataFrame(result).T
result_df.columns = ['의안번호', '처리여부', '의안명', '제안자구분', '제안일자', '의결일자', '의결결과', \
                     '심사진행상태', '문서', '제안회기' , '제안이유', '소관위원회', '제안자']
result_df

,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자
0,2102913,계,지방자치법 일부개정법률안(허영의원 등 10인),의원,2020-08-12,,,접수,,제21대 (2020~2024) 제380회,[제안이유 및 주요내용현행법은 서울특별시ㆍ광역시 및 특별자치시를 제외한 인구 50만...,[],"[허영(더불어민주당/許榮), 강선우(더불어민주당/姜仙祐), 강훈식(더불어민주당/姜勳..."
1,2102912,계,일제강제동원 및 위안부 피해자 인권재단의 설립에 관한 법률안(양정숙의원 등 10인),의원,2020-08-12,,,접수,,제21대 (2020~2024) 제380회,[],[],"[양정숙(무소속/梁貞淑), 김민기(더불어민주당/金敏基), 김수흥(더불어민주당/金洙興..."
2,2102911,계,송유관 안전관리법 일부개정법률안(이규민의원 등 25인),의원,2020-08-12,,,접수,,제21대 (2020~2024) 제380회,[제안이유 및 주요내용2018년 고양시 백석역 인근에서 열수송관 파열사고가 발생함에...,[],"[이규민(더불어민주당/李圭閔), 고영인(더불어민주당/高永寅), 권칠승(더불어민주당/..."
3,2102910,계,재난 및 안전관리 기본법 일부개정법률안(박덕흠의원 등 12인),의원,2020-08-12,,,접수,,제21대 (2020~2024) 제380회,[제안이유 및 주요내용현행법은 각종 재난으로부터 국토를 보존하고 국민의 생명·신체 ...,[],"[박덕흠(미래통합당/朴德欽), 김기현(미래통합당/金起炫), 김도읍(미래통합당/金度邑..."
4,2102909,계,국민연금법 일부개정법률안(류성걸의원 등 13인),의원,2020-08-12,,,접수,,제21대 (2020~2024) 제380회,[제안이유 및 주요내용현행법령에 따르면 보건복지부장관은 5년마다 국민연금 재정 수지...,[],"[류성걸(미래통합당/柳性杰), 구자근(미래통합당/具滋根), 김병욱(미래통합당/金炳旭..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,2102701,계,정부조직법 일부개정법률안(김석기의원 등 19인),의원,2020-08-05,,,소관위접수,의안원문,제21대 (2020~2024) 제380회,"[제안이유현재 재외동포 수가 750만명에 육박하는 가운데, 전 세계에 퍼져있는 재외...",행정안전위원회,"[김석기(미래통합당/金碩基), 강민국(미래통합당/姜旻局), 구자근(미래통합당/具滋根..."
214,2102700,계,농수산물 유통 및 가격안정에 관한 법률 일부개정법률안(이원택의원 등 16인),의원,2020-08-05,,,소관위접수,의안원문,제21대 (2020~2024) 제380회,[제안이유 및 주요내용현행법은 농산물의 유통을 원활하게 하고 적정가격 유지를 위하여...,농림축산식품해양수산위원회,"[이원택(더불어민주당/李源澤), 김경만(더불어민주당/金京萬), 맹성규(더불어민주당/..."
215,2102699,계,조세특례제한법 일부개정법률안(이달곤의원 등 11인),의원,2020-08-05,,,소관위접수,의안원문,제21대 (2020~2024) 제380회,"[제안이유현행법은 쌀시장 개방에 따른 농촌경제의 어려움을 감안하고, 각종 FTA 체...",기획재정위원회,[]
216,2102698,계,지방세특례제한법 일부개정법률안(이달곤의원 등 11인),의원,2020-08-05,,,소관위접수,의안원문\n비용추계서,제21대 (2020~2024) 제380회,"[제안이유현행법은 쌀시장 개방에 따른 농촌경제의 어려움을 감안하고, 각종 FTA 체...",행정안전위원회,[]


In [26]:
result_df.to_csv('21대 국회 의안정보(1).csv', encoding='utf-8-sig', index=False)